# Fine-Tuning a pretrained model PyTorch Trainer

use state-of-the-art models without having to train one from scratch. 🤗 Transformers provides access to thousands of pretrained models for a wide range of tasks. When you use a pretrained model, you train it on a dataset specific to your task. This is known as fine-tuning, an incredibly powerful training technique. In this tutorial, you will fine-tune a pretrained model with a deep learning framework of your choice:

- **Fine-tune a pretrained model with 🤗 Transformers Trainer.**
- Fine-tune a pretrained model in TensorFlow with Keras.
- **Fine-tune a pretrained model in native PyTorch.**


[Hugging Face Datasets Overview Link (PyTorch)](https://youtu.be/_BZearw7f0w)

Before you can fine-tune a pretrained model, download a dataset and prepare it for training. The previous tutorial showed you how to process data for training, and now you get an opportunity to put those skills to the test!


Begin by loading the [Yelp Reviews](https://huggingface.co/datasets/yelp_review_full) dataset:




In [12]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]


{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

As you now know, you need a tokenizer to process the text and include a padding and truncation strategy to handle any variable sequence lengths. To process your dataset in one step, use 🤗 Datasets map method to apply a preprocessing function over the entire dataset:

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 50000/50000 [00:06<00:00, 7761.23 examples/s]


If you like, you can create a smaller subset of the full dataset to fine-tune on to reduce the time it takes:

In [14]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

### **Train**

Now that you have your dataset, you can fine-tune a pretrained model. The first step is to load a pretrained model from the [Model Hub](https://huggingface.co/models). For this tutorial, you will use the [distilbert-base-uncased](https://huggingface.co/distilbert-base-uncased) model. This is a smaller version of the [BERT](https://huggingface.co/bert-base-uncased) model trained on the [BookCorpus](https://huggingface.co/datasets/bookcorpus) and [English Wikipedia](https://huggingface.co/datasets/wikipedia) datasets. The smaller size of the model makes it easier to fine-tune, but it also has less parameters to learn from the small Yelp Reviews dataset.

[Hugging Face Trainer API Link ](https://youtu.be/nvBXf7s7vTI)



In [15]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Next, create a TrainingArguments class which contains all the hyperparameters you can tune as well as flags for activating different training options. For this tutorial you can start with the default training hyperparameters, but feel free to experiment with these to find your optimal settings.

Specify where to save the checkpoints from your training:

In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

Trainer does not automatically evaluate model performance during training. You’ll need to pass Trainer a function to compute and report metrics. The 🤗 Evaluate library provides a simple accuracy function you can load with the evaluate.load (see this quicktour for more information) function:



In [17]:
import numpy as np
import evaluate 

metric = evaluate.load("accuracy")


Call compute on metric to calculate the accuracy of your predictions. Before passing your predictions to compute, you need to convert the predictions to logits (remember all 🤗 Transformers models return logits):

In [18]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)



In [19]:
# if you'd like to monitor your evaluation metrics during fine-tuning 
# specify the evaluation_strategy parameter in your training arguments to 
# report the eval metric at the end of each epoch

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")


In [20]:
# Create a Trainer object with your model, 
# training arguments, training and test datasets, and evaluation function:

trainer = Trainer(
    model=model,
    args=training_args, 
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# train that mf model

trainer.train()
